# First Install the dependencies and required libraries & FIles

In [ ]:
# Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install text-unidecode
!pip install matplotlib>=3.3.2

## Install NeMo
BRANCH = 'r2.0.0rc0'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

## Grab the config we'll use in this example
!mkdir configs
!wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/citrinet/config_bpe.yaml

"""
Remember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!
Alternatively, you can uncomment the exit() below to crash and restart the kernel, in the case
that you want to use the "Run All Cells" (or similar) option.
"""
# exit()


In [ ]:
!git clone https://github.com/7egment/pyannote-3.1-offline
!pip install pyannote-audio==3.1.1

# We will edit the path on config file to work offline

In [3]:
# Update the paths in the configuration file
!sed -i 's|/home/e6quisitory/pyannote-3.1-offline/seg.bin|pyannote-3.1-offline/seg.bin|g' /content/pyannote-3.1-offline/config.yaml
!sed -i 's|/home/e6quisitory/pyannote-3.1-offline/vox.bin|pyannote-3.1-offline/vox.bin|g' /content/pyannote-3.1-offline/config.yaml




In [4]:
# NeMo's "core" package
import nemo
# NeMo's ASR collection - this collections contains complete ASR models and
# building blocks (modules) for ASR
import nemo.collections.asr as nemo_asr

# FInally Inference Script , Dont Forget to add MetanoiaLabsModel from ``\FinalinferenceNotebook`` directory

In [5]:
import os
import json
from nemo.collections.asr.models import EncDecCTCModelBPE
from pyannote.audio import Pipeline
from pydub import AudioSegment

# Initialize the ASR model
asr_model = EncDecCTCModelBPE.restore_from(restore_path="/content/MetanoiaLabsModel.nemo")

# Initialize the diarization pipeline
pipeline = Pipeline.from_pretrained("pyannote-3.1-offline/config.yaml")

# Directory containing WAV files
audio_dir = "./"      # <-----------------------  Put wav Folder Here

# Directory to save the JSON files
output_dir = "./Finaljsonfiles"

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# List all WAV files in the directory
audio_files = [os.path.join(audio_dir, f) for f in os.listdir(audio_dir) if f.endswith('.wav')]

# Function to transcribe a segment
def transcribe_segment(segment_audio):
    return asr_model.transcribe([segment_audio], batch_size=1)[0]

# Function to handle audio segment extraction and transcription
def process_audio_file(audio_file):
    audio_id = os.path.basename(audio_file).split('.')[0]
    json_output = []

    try:
        # Load the audio file using pydub
        audio = AudioSegment.from_wav(audio_file)

        # Diarize the audio file
        diarization = pipeline(audio_file)

        # Extract and transcribe each segment
        for turn, _, speaker in diarization.itertracks(yield_label=True):
            start_time = turn.start
            end_time = turn.end

            # Check if the segment duration is valid
            if end_time <= start_time:
                print(f"Skipping invalid segment from {start_time} to {end_time} in {audio_file}")
                continue

            # Extract the audio segment using pydub
            segment_audio = audio[start_time * 1000:end_time * 1000]  # pydub works in milliseconds

            # Transcribe the audio segment
            # Save segment to a temporary file
            segment_path = f"{audio_id}_segment_{start_time:.3f}_{end_time:.3f}.wav"
            segment_audio.export(segment_path, format="wav")

            try:
                text = transcribe_segment(segment_path)
            except Exception as e:
                print(f"Transcription failed for segment {segment_path}: {e}")
                text = ""

            # Append the transcription to the JSON output
            json_output.append({
                "start": start_time,
                "end": end_time,
                "speaker": speaker,
                "text": text
            })

            # Clean up the segment audio file
            os.remove(segment_path)
    except Exception as e:
        print(f"Error processing {audio_file}: {e}")

    # Save the JSON output to a file
    json_file = os.path.join(output_dir, f"{audio_id}.json")
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(json_output, f, ensure_ascii=False, indent=4)

    print(f"Processed and saved {json_file}")

# Process each audio file
for audio_file in audio_files:
    process_audio_file(audio_file)

print("All files processed.")


[NeMo W 2024-07-22 21:29:31 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
      torchaudio.set_audio_backend("soundfile")
    


[NeMo I 2024-07-22 21:29:34 mixins:172] Tokenizer SentencePieceTokenizer initialized with 64 tokens


[NeMo W 2024-07-22 21:29:34 modelPT:176] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /content/Last_Hope.json
    sample_rate: 16000
    batch_size: 16
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    num_workers: 8
    pin_memory: true
    is_tarred: false
    tarred_audio_filepaths: null
    shard_strategy: scatter
    shuffle_n: 2048
    bucketing_strategy: synced_randomized
    bucketing_batch_size: null
    
[NeMo W 2024-07-22 21:29:34 modelPT:183] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /content/Last_Hope.json
    sample_rate: 16000
    batch_size: 16
    shuffle: false
    num_work

[NeMo I 2024-07-22 21:29:34 features:305] PADDING: 16
[NeMo I 2024-07-22 21:29:35 save_restore_connector:263] Model EncDecCTCModelBPE was successfully restored from /content/MetanoiaLabsModel.nemo.


Transcribing: 100%|██████████| 1/1 [00:00<00:00, 28.89it/s]

Processed and saved ./Finaljsonfiles/audio_sample_12.json
All files processed.
